# Hyperparameter Search
In this notebook, we take a look at using hyperparameter search to automate the process of selecting/fine tuning the hyperparameters used in our neural network. The parameters of a neural network are the weights and biases that are learned and continually updated during training. Networks often have thousands or even millions of parameters, and they are never adjusted by hand, but rather through some sort of automated process like backpropagation. In contrast, hyperparameters effect the architecture of the network itself (they can sort of be thought of as meta-parameters), and are often decided by humans. Things like number of layers, number of neurons per layer, and the learning rate or momentum of the network are all hyperparameters. 

Training neural networks can be very difficult and it is often the role of the programmer to select the hyperparameters that will yield the best results. It is not uncommon for a programmer to run 10 to 20 (or more) experiments, each with different hyperparameters, before arriving at an optimal solution (or even one that is simply "good enough").

Hyperparameter search (often called Hyperparameter Optimization) is a method used to automate the discovery of effective hyperparameters for a network. Rather than using intuition and experience to fine-tune your hyperparameters by hand, hyperparameter search can be used to automatically discover optimal hyperparameters given enough compute time and resources.

In this example we are going to use hyperparameter search to discover a good network architecture and appropriate learning rate, etc... for a classification task using the popular [MNIST dataset](https://en.wikipedia.org/wiki/MNIST_database). We will be using the Keras library to construct our network, and [hyperas](https://github.com/maxpumperla/hyperas), a keras port of the [hyperopt](http://hyperopt.github.io/hyperopt/) library, to automate our hyperparameters.



In [1]:
import dill as pickle
import pprint as pprint
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional

Using TensorFlow backend.


Hyperas syntax is similar to that of a templating engine. It uses double curly-braces (`{{`) to denote hyperparameter content that should be replaced by hyperas at runtime. More on that soon, but for now it is important to realized that when you use hyperas, your code is pre-processed and many iterations of your source code are generated and run with the contents of each `{{...}}` replaced with a different set of hyperparameters each time.

## Data() and Model()

In order for the hyperas optmizer (`keras.optim`) to generate and evaluate different permutations of your source code it the must be given two special functions: `data()` and `model(X_train, Y_train, X_test, Y_test)`. 

`data()` is used to load and preprocess your data and must return `X_train, y_train, X_test, y_test`. This function is run only once by `keras.optim` to avoid reloading your data. To the best of my knowledge, templated `{{` statements are not permitted (or are at least not useful) inside of `data()`.

`model(X_train, Y_train, X_test, Y_test)` is run once per an evaluation of a new hyperparameter combination. This function is responsible for generating a model, training that model (in our case using `keras.fit(...)`), and returning a special dictionary object used by hyperas to evaluate the success of the hyperparameters chosen in that particular configuration (more on this dictionary below).

In [2]:
# Note: For whatever reason, I've experienced a bug with hyperas that
# prevents me from using any kind of comment in either the data() or
# model() function. For this reason I will attempt to describe the 
# code in both of these functions through comments and explanations
# outside of the functions themselves.
def data():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = X_train.reshape(60000, 784)
    X_test = X_test.reshape(10000, 784)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    nb_classes = 10
    y_train = to_categorical(y_train, nb_classes)
    y_test = to_categorical(y_test, nb_classes)
    return X_train, y_train, X_test, y_test

In [21]:
def model(X_train, y_train, X_test, y_test):
    model = Sequential()
    model.add(Dense(512, input_dim=784))
    model.add(Activation('relu'))
    model.add(Dense({{choice([256, 512, 1024])}}))
    model.add(Dropout({{uniform(0.25, 0.75)}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dense({{choice([256, 512, 1024])}}))
    model.add(Dropout({{uniform(0.25, 0.75)}}))
    
    if conditional({{choice(['three', 'four']) == 'four'}}):
        model.add(Dense({{choice([64, 128, 256])}}))
        model.add(Dropout(0.5))
        
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer={{choice(['rmsprop', 'adam', 'sgd'])}})
    
    model.fit(X_train, y_train,
              nb_epoch=1,
              show_accuracy=True,
              verbose=0,
              validation_data=(X_test, y_test))
    
    score, acc = model.evaluate(X_test, y_test, verbose=0)
    return {'loss': -acc, 'status': STATUS_OK}
    

Here we've used three of the most common `hyperas` distribution functions: `choice`, `uniform`, and `conditional`. `choice([a, b, c, ...])` takes a list of possible values from which to select during hyperparameter search. `uniform(min, max)` samples from a uniform distribution between floats `min` and `max`. `conditional()` returns `True` or `False` based on the boolean contents inside of `{{}}` and can be used with `choice()` to easily include or exclude entire code blocks. For a complete list of hyperas (really hyperopt) distrobutions see [here](https://github.com/maxpumperla/hyperas/blob/master/hyperas/distributions.py).

The dictionary returned by `model(...)` must include two required key => value pairs: `loss`, and `status`. `loss` is the value you are trying to minimize through your hyperparameter search. Think of this as the value you are looking to produce given optimal hyperparameters. In our case we are looking for the hyperparameters that give us the highest accuracy in an MNIST classification problem. Because hyperas requires the `loss` value to be minimized during search, and we are looking to maximize accuracy, we invert the sign of our accuracy when assigning it to the value of the loss key in the returned dictionary. Thus we convert `acc` to `-acc`. 

The value of `status` is simply a constant representing whether or not the model should be considered as successful and included in our record of trials. Unless there is an error that is caught or some anomoly in your `model(...)` function this value can be `STATUS_OK`. In addition to these two required keys, you are free to add as many additional key => value pairs to the returned dictionary as you like, essentially making variables local to the `model(...)` function available to the `Trials()` object used in `keras.optim`. A common extra key to use is `model` that allows you to return a reference to the trained model itself.

__Note__: `choice()` and `uniform()` are inside of the `{{}}` but with `conditional()` these brackets are inside the function parenthesis. 

## Running the search 

Once we've created our `data()` and `model(...)` functions, we are ready to execute our hyperparameter search using the `keras.optim.minimize(...)` function. This function performs `max_evals` versions of your `model(...)` function using different combinations of hyperparameter values attempting to minimize the "loss" value in the dictionary returned by each internal call to `model(...)`. It returns a tuple of the `(score, accuracy)` of the best run as well as the Keras model that achieved that run.

__WARNING__: Performing a hyperparameter search in this way can take a __very long time__. The search space grows exponentially with each addition of a `hyperas` distribution function. The `max_evals` named parameter to `keras.optim.minimize(...)` can be used to limit the maximum number of hyperparameter searches that will be run. Also note that keeping the number of epochs low for each model fitting (i.e. `model.fit(X, y, nb_epoch=1)`) can drastically reduce the time it takes to execute the hyperparameter searches. The danger with this strategy is that hyperparameters, like learning rate, that take longer to achieve good results will be severly disadvantaged by the search and an optimal solution may be overlooked.

In [29]:
trials = Trials()
best_run, foo = optim.minimize(model=model,
                          data=data,
                          algo=tpe.suggest,
                          max_evals=1000,
                          notebook_name='hyperparameter_search',
                          verbose=False,
                          trials=trials)

print("Best run scored was")
print(best_run)

# serialize and save the trials object
try:
    pickle.dump(trials.trials, open('../data/hyperparameter_trials.pickel', 'wb'), recurse=True, byref=False)
except e:
    print(e)
    

/usr/local/lib/python3.5/dist-packages/keras/models.py:635: UserWarning: The "show_accuracy" argument is deprecated, instead you should pass the "accuracy" metric to the model at compile time:
`model.compile(optimizer, loss, metrics=["accuracy"])`
  warnings.warn('The "show_accuracy" argument is deprecated, '


Train on 60000 samples, validate on 10000 samples
Epoch 1/1
5s - loss: 0.2475 - acc: 0.9324 - val_loss: 0.2196 - val_acc: 0.9528
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
6s - loss: 0.2891 - acc: 0.9140 - val_loss: 0.1190 - val_acc: 0.9659
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
4s - loss: 0.2600 - acc: 0.9272 - val_loss: 0.1242 - val_acc: 0.9687
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
5s - loss: 0.2253 - acc: 0.9324 - val_loss: 0.1093 - val_acc: 0.9655
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
6s - loss: 0.2341 - acc: 0.9303 - val_loss: 0.1305 - val_acc: 0.9617
Best run scored was
{'Dense_1': 2, 'Activation': 0, 'Dense_2': 0, 'Dense': 0, 'optimizer': 0, 'Dropout_1': 0.2676392100975012, 'Dropout': 0.3428208352348263}


In [25]:
file = open('../data/hyperparameter_trials.pickel', 'rb')
t = pickle.load(file)